## Test Run 3: Refine Labelling Function

This notebook continues from Test Run 2, but addresses the following issues:

1. Load more papers than toy examples as the toy Test Run 2. 
2. Complete/refine labelling functions.
3. Load two evaluation datasets and calculate overlap: (1) 5 system/model/method papers + 5 empirical papers; (2) 20K labelled paper dataset.



In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))
debug_mode=1 # if not, debug_mode=0

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

from snorkel import SnorkelSession
from snorkel.parser.spacy_parser import Spacy
from snorkel.parser import CorpusParser
from snorkel.models import Document, Sentence

session = SnorkelSession()
print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

sents = session.query(Sentence).all()
n_max_corpus=0
for sent in sents:
    n_max_corpus=max(n_max_corpus,len(sent.words))

print("The longest sentence has "+str(n_max_corpus)+" tokens.")

from snorkel.models import Document
from util import number_of_people

docs = session.query(Document).all()

train_sents = set()
dev_sents   = set()
test_sents  = set()

for i, doc in enumerate(docs):
    for s in doc.sentences:
        if i % 10 == 8 and "cscw18"!=doc.name[:6]:
            dev_sents.add(s)
#         elif i % 10 == 9:
        elif "cscw18"==doc.name[:6]:  # replace the earlier 10% test documents as cscw'18 annotation guideline 10 examples
            test_sents.add(s)
        elif "cscw18"!=doc.name[:6]:
            train_sents.add(s)
            
print(len(test_sents))


Documents: 5560
Sentences: 19540
The longest sentence has 157 tokens.
68


In [3]:
from snorkel.models import candidate_subclass
from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.matchers import *

ngrams = Ngrams(n_max=n_max_corpus) # we define the maximum n value as n_max_corpus
document_breakdown_map=dict() # maps doc_id into a dict of ["Background", "Purpose", "Mechanism", "Method", "Finding"]
test_doc_breakdown_map=dict()
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
    

def extract_and_display(matcher,candidate_class,candidate_class_name,document_breakdown_map=None,selected_split=0,is_print=True):  # split over train/dev/test but returns only train set
#     input(candidate_class)
    for (i, sents) in ([(0,train_sents), (1,dev_sents), (2,test_sents)] if selected_split==0 else ([(2,test_sents)] if selected_split==2 else [(1,dev_sents)])):
        %time matcher.apply(sents, split=i)
        printmd("**Split "+str(i)+" - number of candidates extracted: "+str(session.query(candidate_class).filter(candidate_class.split == i).count())+"**\n\n")
    cands = session.query(candidate_class).filter(candidate_class.split == selected_split).all()
    if is_print:
        for i in range(min(4,len(cands))): # to print all cands, range(len(cands))
            printmd("**"+str(i)+"/"+str(len(cands))+" Candidate/Span:**\t`"+str(cands[i])+"`")
            printmd("**Its parent Sentence's text:**\t"+str(cands[i].get_parent().text))
            printmd("**Its parent Document's text:**\t"+str(cands[i].get_parent().get_parent().__dict__))
            print() 
        
    for cand in cands:
#         print(cand.__dict__)
        doc_name=cand.get_parent().get_parent().name
        if doc_name not in document_breakdown_map:
            document_breakdown_map[doc_name]=dict()
        if candidate_class_name not in document_breakdown_map[doc_name]:
            document_breakdown_map[doc_name][candidate_class_name]=[]
        document_breakdown_map[doc_name][candidate_class_name]+=[cand]
        
    cands = session.query(candidate_class).filter(candidate_class.split == 2).all()
    for cand in cands:
        print(cand.__dict__)
        doc_name=cand.get_parent().get_parent().name
        if doc_name not in test_doc_breakdown_map:
            test_doc_breakdown_map[doc_name]=dict()
        if candidate_class_name not in test_doc_breakdown_map[doc_name]:
            test_doc_breakdown_map[doc_name][candidate_class_name]=[]
        test_doc_breakdown_map[doc_name][candidate_class_name]+=[cand]
    
    
        
    return cands
    

We want to differentiate ``Background`` with ``Purpose``. Recall our ``Background`` definition from Google Doc:

``Contains words that indicate prior work (e.g., “traditionally”, “researchers have…”), and then following sentence/span starts with some variant of “In this paper, we introduce…” (Exploring adjacency relationship through helper function in Snorkel)``


In [6]:
# remove if necessary 
# session.query(Background).all()
import snorkel.models.candidate as candidate

def del_defined_candidate_class(class_name):
    print("Existing", candidate.candidate_subclasses)
    del(candidate.candidate_subclasses[class_name])
    print("After deletion", candidate.candidate_subclasses)
    
## Usage:    
# del_defined_candidate_class('Purpose')

non_comma_matcher=DictionaryMatch(d=[','],longest_match_only=True,reverse=True)  

The next few cells describe our ``Span Matching`` phase: we respectively define the `Span` for the 5 segments are , and centrally stored in a document-id-indexed hashmap. This helps us to group the 5 segments back into documents.
3. Only those documents that have at least 3 segments are considered as valid

Following this ``Span Matching`` phase, we will be having an ``Document Aggregation`` phase, where adjacency relationship and (or) waterfall models are being emphasized through multiple criteria. For example, the following criteria prevents non-abstracts (e.g. proceeding cover letter, tutorial introduction, etc.) or less well-strctured abstracts from being included. 

1. Only those documents that have all 5 segments are considered as valid
2. Only those documents that have at least 4 segments are cosidered as valid

In [60]:
# Compound Matcher for Background: 
Background = candidate_subclass('Background', ['background_cue'])
transition_word=DictionaryMatch(d=['while','unlike','despite'],longest_match_only=True) 
transition_prev_work=DictionaryMatch(d=['previous','earlier','past'],longest_match_only=True) 
dict_background_matcher=DictionaryMatch(d=['previous work','traditionally','researchers'],longest_match_only=True) 
excluded_dict_background_matcher=DictionaryMatch(d=['we','unlike','our'],longest_match_only=True,reverse=True) 
non_comma_dict_background_matcher=CandidateExtractor(Background, [ngrams], [Intersection(non_comma_matcher,Union(dict_background_matcher,Intersection(transition_word,transition_prev_work)),excluded_dict_background_matcher)])
background_cands=extract_and_display(non_comma_dict_background_matcher,Background,"Background",document_breakdown_map)
# print(document_breakdown_map)

<class 'snorkel.models.candidate.Background'>
Clearing existing...
Running UDF...
[===                                     ] 7%

KeyboardInterrupt: 

**Split 0 - number of candidates extracted: 0**



Clearing existing...
Running UDF...
[=                                       ] 0%

KeyboardInterrupt: 

**Split 1 - number of candidates extracted: 0**



Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 480 ms, sys: 63.1 ms, total: 543 ms
Wall time: 526 ms


**Split 2 - number of candidates extracted: 0**



In [46]:
# Compound Matcher for Purpose:
Purpose=candidate_subclass('Purpose',['purpose_cue'])

transition_regex_matcher=RegexMatchSpan(rgx="((^|\s)however.*$)|((^|\s)but(?!(also))*$)",longest_match_only=True)  # Correction: purpose 
excluded_dict_purpose_matcher=SentenceMatch(d=['but also','but without','but sometimes'],longest_match_only=True,reverse=True)  # the parent sentence shall not include "but also"
transition_matcher=Intersection(transition_regex_matcher,excluded_dict_purpose_matcher)

comparative_degree_matcher=Intersection(RegexMatchSpan(rgx="(.*more.*than.*$)|(.*er than.*$)",longest_match_only=True),transition_prev_work)  # Correction: purpose 
other_regex_matcher=RegexMatchSpan(rgx="(.*extend.*$)|(.*offer.*$)",longest_match_only=True)

dict_purpose_matcher=DictionaryMatch(d=['in this paper','in the paper',' that can ','in this study','to examine','we examine','to investigate','implications'],longest_match_only=True) 

# Unit test
# non_comma_dict_purpose_matcher=CandidateExtractor(Purpose, [ngrams], [Intersection(non_comma_matcher,other_regex_matcher)])

non_comma_dict_purpose_matcher=CandidateExtractor(Purpose, [ngrams], [Intersection(non_comma_matcher,Union(comparative_degree_matcher,other_regex_matcher,dict_purpose_matcher,transition_matcher))]) #,intersection(excluded_dict_purpose_matcher,transition_regex_matcher)])
purpose_cands=extract_and_display(non_comma_dict_purpose_matcher,Purpose,"Purpose",document_breakdown_map)
# print(document_breakdown_map)

<class 'snorkel.models.candidate.Purpose'>
Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 1min 35s, sys: 1.38 s, total: 1min 37s
Wall time: 1min 38s


**Split 0 - number of candidates extracted: 1501**



Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 11.8 s, sys: 468 ms, total: 12.3 s
Wall time: 12.2 s


**Split 1 - number of candidates extracted: 203**



Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 761 ms, sys: 117 ms, total: 878 ms
Wall time: 878 ms


...

**Its parent Document's text:**	{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x111488240>, 'name': '5834868425ff05a97b00e0fe', 'stable_id': '5834868425ff05a97b00e0fe::document:0:0', 'meta': {'file_name': '70kpaper_061418_cleaned_noBookLecture.tsv'}, 'id': 315, 'type': 'document', 'sentences': [Sentence(Document 5834868425ff05a97b00e0fe,0,b'Modern computing paradigms have frequently adopted concepts from distributed systems.'), Sentence(Document 5834868425ff05a97b00e0fe,1,b'The quest for scalability, reliability and cost reduction has led to the development of massively distributed systems, which extend organisational boundaries.'), Sentence(Document 5834868425ff05a97b00e0fe,2,b'Voluntary computing environments (such as BOINC), Grids (such as EGEE and Globus), and more recently Cloud Computing (both open source and commercial) have established themselves as a range of distributed systems.\n')]}

**3/1501 Candidate/Span:**	`Purpose(Span("b'however'", sentence=12099, chars=[5,11], words=[2,2]))`

**Its parent Sentence's text:**	Few, however, allow new interfaces to be created from scratch because they do not provide a means of demonstrating when a recorded macro should be invoked.

**Its parent Document's text:**	{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1122fb828>, 'name': '5834868725ff05a97b013a8a', 'stable_id': '5834868725ff05a97b013a8a::document:0:0', 'meta': {'file_name': '70kpaper_061418_cleaned_noBookLecture.tsv'}, 'id': 1836, 'type': 'document', 'sentences': [Sentence(Document 5834868725ff05a97b013a8a,0,b'Many programming by demonstration (PBD) systems elaborate on the idea of macro recording, and they allow users to extend existing applications.'), Sentence(Document 5834868725ff05a97b013a8a,1,b'Few, however, allow new interfaces to be created from scratch because they do not provide a means of demonstrating when a recorded macro should be invoked.'), Sentence(Document 5834868725ff05a97b013a8a,2,b'This paper discusses stimulus-response systems that allow both the when (stimulus/event) and the what (response macro) to be demonstrated.\n')]}


10
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1a21999fd0>, 'type': 'purpose', 'purpose_cue_cid': None, 'purpose_cue_id': 26962, 'split': 2, 'id': 1705}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1a21990080>, 'type': 'purpose', 'purpose_cue_cid': None, 'purpose_cue_id': 26963, 'split': 2, 'id': 1706}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1a219900f0>, 'type': 'purpose', 'purpose_cue_cid': None, 'purpose_cue_id': 26964, 'split': 2, 'id': 1707}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1a21990160>, 'type': 'purpose', 'purpose_cue_cid': None, 'purpose_cue_id': 26965, 'split': 2, 'id': 1708}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1a219901d0>, 'type': 'purpose', 'purpose_cue_cid': None, 'purpose_cue_id': 26966, 'split': 2, 'id': 1709}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1a21990240>, 'type': 'purpose', 'purpose

In [7]:
# Compound Matcher for Mechanism: 
Mechanism = candidate_subclass('Mechanism', ['mechanism_cue']) 
dict_mechanism_matcher=DictionaryMatch(d=['introduce','introduces','propose','proposes','we propose','we develop','approach'],longest_match_only=True) 
non_comma_dict_mechanism_matcher=CandidateExtractor(Mechanism, [ngrams], [Intersection(non_comma_matcher,dict_mechanism_matcher)])
mechanism_cands=extract_and_display(non_comma_dict_mechanism_matcher,Mechanism,"Mechanism",document_breakdown_map)

Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 54.1 s, sys: 852 ms, total: 54.9 s
Wall time: 55.7 s


**Split 0 - number of candidates extracted: 1404**



Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 7.21 s, sys: 422 ms, total: 7.63 s
Wall time: 7.52 s


**Split 1 - number of candidates extracted: 169**



Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 475 ms, sys: 103 ms, total: 578 ms
Wall time: 609 ms


...

**Its parent Document's text:**	{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x10fc3b9e8>, 'name': '5834868825ff05a97b0175d2', 'stable_id': '5834868825ff05a97b0175d2::document:0:0', 'meta': {'file_name': '70kpaper_061418_cleaned_noBookLecture.tsv'}, 'id': 3472, 'type': 'document', 'sentences': [Sentence(Document 5834868825ff05a97b0175d2,0,b'Abstract\xe2\x80\x93'), Sentence(Document 5834868825ff05a97b0175d2,1,b'We describe a new set of prismatic movement primitives for cubic modular robots.'), Sentence(Document 5834868825ff05a97b0175d2,2,b'Our approach appears more practical than previous metamodule-based approaches.'), Sentence(Document 5834868825ff05a97b0175d2,3,b"We also describe recent hardware developments in our cu-bic robot modules that have sufficient stiffness and actuator strength so that when they work together they can realize, in earth's gravity, all of the motion primitives we describe here.\n")]}

**3/1404 Candidate/Span:**	`Mechanism(Span("b'In contrast to existing approaches that rely only on computer vision'", sentence=19748, chars=[0,67], words=[0,10]))`

**Its parent Sentence's text:**	In contrast to existing approaches that rely only on computer vision, we propose an alternative method for improving perception by learning from human teammates.

**Its parent Document's text:**	{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x11b341208>, 'name': '5834868825ff05a97b01791b', 'stable_id': '5834868825ff05a97b01791b::document:0:0', 'meta': {'file_name': '70kpaper_061418_cleaned_noBookLecture.tsv'}, 'id': 4036, 'type': 'document', 'sentences': [Sentence(Document 5834868825ff05a97b01791b,0,b'In robotics research, perception is one of the most challenging tasks.'), Sentence(Document 5834868825ff05a97b01791b,1,b'In contrast to existing approaches that rely only on computer vision, we propose an alternative method for improving perception by learning from human teammates.'), Sentence(Document 5834868825ff05a97b01791b,2,b'To evaluate, we apply this idea to a door detection problem.'), Sentence(Document 5834868825ff05a97b01791b,3,b'A set of preliminary experiments has been completed using software agents with real vision data.'), Sentence(Document 5834868825ff05a97b01791b,4,b'Our results demonstrate that information inferred from teammate observations significantly improves the perception precision.\n')]}


{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1a1f7d7be0>, 'type': 'mechanism', 'mechanism_cue_cid': None, 'id': 3157, 'split': 2, 'mechanism_cue_id': 26962}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1a1f7d7ef0>, 'type': 'mechanism', 'mechanism_cue_cid': None, 'id': 3158, 'split': 2, 'mechanism_cue_id': 28446}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1a1f7d7eb8>, 'type': 'mechanism', 'mechanism_cue_cid': None, 'id': 3159, 'split': 2, 'mechanism_cue_id': 28447}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1a1f7d0908>, 'type': 'mechanism', 'mechanism_cue_cid': None, 'id': 3160, 'split': 2, 'mechanism_cue_id': 28448}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1a1f7d0ac8>, 'type': 'mechanism', 'mechanism_cue_cid': None, 'id': 3161, 'split': 2, 'mechanism_cue_id': 28449}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1a1f7d07f0>, 

In [8]:
# TODO: Compound Matcher for Method: 
Method = candidate_subclass('Method', ['method_cue'])

dict_method_matcher=DictionaryMatch(d=['dataset','benchmark','experiment ','experiments',"empirical","participant","survey"," conduct"," analyze"],longest_match_only=True) 

non_comma_dict_method_matcher=CandidateExtractor(Method, [ngrams], [Intersection(non_comma_matcher,dict_method_matcher)])
method_cands=extract_and_display(non_comma_dict_method_matcher,Method,"Method",document_breakdown_map)


Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 53 s, sys: 864 ms, total: 53.9 s
Wall time: 54.4 s


**Split 0 - number of candidates extracted: 752**



Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 6.82 s, sys: 364 ms, total: 7.18 s
Wall time: 6.96 s


**Split 1 - number of candidates extracted: 81**



Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 428 ms, sys: 71.6 ms, total: 499 ms
Wall time: 464 ms


...

**Its parent Document's text:**	{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x11b2b7e80>, 'name': '5834868825ff05a97b017851', 'stable_id': '5834868825ff05a97b017851::document:0:0', 'meta': {'file_name': '70kpaper_061418_cleaned_noBookLecture.tsv'}, 'id': 3743, 'type': 'document', 'sentences': [Sentence(Document 5834868825ff05a97b017851,0,b'The Metaphor is intended to contribute to the Agile Programming value of communication.'), Sentence(Document 5834868825ff05a97b017851,1,b'Previously, some of the author studied the Metaphor as a means of communication among team members and between them and clients.'), Sentence(Document 5834868825ff05a97b017851,2,b"This paper examines the Metaphor's contribution to the software architecture."), Sentence(Document 5834868825ff05a97b017851,3,b'Both experiments seemto reveal that the Metaphor has poor effectiveness.\n')]}

**3/752 Candidate/Span:**	`Method(Span("b'The commonality amongst the experiments has permitted the ability to objectively...\n'", sentence=21988, chars=[0,83], words=[0,12]))`

**Its parent Sentence's text:**	The commonality amongst the experiments has permitted the ability to objectively...


**Its parent Document's text:**	{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x11b453c88>, 'name': '59364828c50f90cdd3aca7f8', 'stable_id': '59364828c50f90cdd3aca7f8::document:0:0', 'meta': {'file_name': '70kpaper_061418_cleaned_noBookLecture.tsv'}, 'id': 4676, 'type': 'document', 'sentences': [Sentence(Document 59364828c50f90cdd3aca7f8,0,b'This paper presents an experimental evaluation and comparison of basic strategies that have been proposed for force control of robot manipulators.'), Sentence(Document 59364828c50f90cdd3aca7f8,1,b'This experimental review of force control methodologies is unique in its breadth--never has such a complete spectrum of strategies been experimentally compared on the same system.'), Sentence(Document 59364828c50f90cdd3aca7f8,2,b'The commonality amongst the experiments has permitted the ability to objectively...\n')]}


{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1a1fa40710>, 'type': 'method', 'method_cue_cid': None, 'id': 4000, 'split': 2, 'method_cue_id': 26962}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1a1fa40b00>, 'type': 'method', 'method_cue_cid': None, 'id': 4001, 'split': 2, 'method_cue_id': 29212}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1a1fa40ba8>, 'type': 'method', 'method_cue_cid': None, 'id': 4002, 'split': 2, 'method_cue_id': 29213}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1a1fa40748>, 'type': 'method', 'method_cue_cid': None, 'id': 4003, 'split': 2, 'method_cue_id': 29214}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1a1fa404e0>, 'type': 'method', 'method_cue_cid': None, 'id': 4004, 'split': 2, 'method_cue_id': 29215}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1a1fa49400>, 'type': 'method', 'method_cue_cid': None, 'id

In [9]:
# TODO: Compound Matcher for Finding: 
Finding = candidate_subclass('Finding', ['finding_cue'])

dict_finding_matcher=DictionaryMatch(d=['show that','shows that','found','indicate','results','performance','find'],longest_match_only=True) 

non_comma_dict_finding_matcher=CandidateExtractor(Finding, [ngrams], [Intersection(non_comma_matcher,dict_finding_matcher)])
finding_cands=extract_and_display(non_comma_dict_finding_matcher,Finding,"Finding",document_breakdown_map)

Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 56.2 s, sys: 803 ms, total: 57 s
Wall time: 57.5 s


**Split 0 - number of candidates extracted: 1753**



Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 6.98 s, sys: 347 ms, total: 7.32 s
Wall time: 7.12 s


**Split 1 - number of candidates extracted: 176**



Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 469 ms, sys: 94.4 ms, total: 564 ms
Wall time: 529 ms


...

**Its parent Document's text:**	{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x11b4e66a0>, 'name': '59382e72c50f90cdd3ada1ca', 'stable_id': '59382e72c50f90cdd3ada1ca::document:0:0', 'meta': {'file_name': '70kpaper_061418_cleaned_noBookLecture.tsv'}, 'id': 5003, 'type': 'document', 'sentences': [Sentence(Document 59382e72c50f90cdd3ada1ca,0,b'We describe our current work in developing novel mechanisms for managing security and privacy in pervasive computing environments.'), Sentence(Document 59382e72c50f90cdd3ada1ca,1,b'More specifically, we have developed and evaluated three different applications, including a contextual instant messenger, a people finder application, and a phone-based application for access control.'), Sentence(Document 59382e72c50f90cdd3ada1ca,2,b'We also draw out some themes we have learned thus far for user-controllable security and privacy.\n')]}

**3/1753 Candidate/Span:**	`Finding(Span("b'We show on both generated data and on real data from the first 169 match runs of the UNOS nationwide kidney exchange that even a very small number of non-adaptive edge queries per vertex results in large gains in expected successful matches.\n'", sentence=24335, chars=[0,241], words=[0,45]))`

**Its parent Sentence's text:**	We show on both generated data and on real data from the first 169 match runs of the UNOS nationwide kidney exchange that even a very small number of non-adaptive edge queries per vertex results in large gains in expected successful matches.


**Its parent Document's text:**	{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x11b5852b0>, 'name': '595d391dc50f90cdd3bdeda2', 'stable_id': '595d391dc50f90cdd3bdeda2::document:0:0', 'meta': {'file_name': '70kpaper_061418_cleaned_noBookLecture.tsv'}, 'id': 5357, 'type': 'document', 'sentences': [Sentence(Document 595d391dc50f90cdd3bdeda2,0,b'We empirically explore the application of (adaptations of) these algorithms to the kidney exchange problem, where patients with end-stage renal failure swap willing but incompatible donors.'), Sentence(Document 595d391dc50f90cdd3bdeda2,1,b'We show on both generated data and on real data from the first 169 match runs of the UNOS nationwide kidney exchange that even a very small number of non-adaptive edge queries per vertex results in large gains in expected successful matches.\n')]}


{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1a1ff899e8>, 'type': 'finding', 'id': 5939, 'split': 2, 'finding_cue_cid': None, 'finding_cue_id': 30865}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1a1ff89a58>, 'type': 'finding', 'id': 5940, 'split': 2, 'finding_cue_cid': None, 'finding_cue_id': 30866}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1a1ff82f28>, 'type': 'finding', 'id': 5941, 'split': 2, 'finding_cue_cid': None, 'finding_cue_id': 29216}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1a1ff82ef0>, 'type': 'finding', 'id': 5942, 'split': 2, 'finding_cue_cid': None, 'finding_cue_id': 30867}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1a1ff828d0>, 'type': 'finding', 'id': 5943, 'split': 2, 'finding_cue_cid': None, 'finding_cue_id': 30868}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1a1ff82f60>, 'type': 'finding', 'id': 5944,

Next is the ``Document Aggregation`` phase. First we show a few document examples, of how ``document_breakdown_map`` looks like, as well as how do we aggregate. 

In [13]:
for ind,docid in enumerate(document_breakdown_map.keys()):
    print("\n\nDocument "+str(ind)+"/"+str(len(document_breakdown_map.keys())))
    print("Extracted segments\n\n",docid,document_breakdown_map[docid],"\n\n")
    print("Complete abstract\n\n",document_breakdown_map[docid][list(document_breakdown_map[docid].keys())[0]][0].get_parent().get_parent().sentences)
    if ind>2:
        break



Document 0/3039
Extracted segments

 59398333c50f90cdd3ae2761 {'Background': [Background(Span("b'recognized the need for a visualization tool that would allow researchers to examine and evaluate specific word correspondences generated by a translation system.'", sentence=24132, chars=[83,244], words=[13,36]))], 'Purpose': [Purpose(Span("b'we recognized the need for a visualization tool that would allow researchers to examine and evaluate specific word correspondences generated by a translation system.'", sentence=24132, chars=[80,244], words=[12,36]))], 'Mechanism': [Mechanism(Span("b'We developed Cairo to fill this need.'", sentence=24133, chars=[0,36], words=[0,7]))]} 


Complete abstract

 [Sentence(Document 59398333c50f90cdd3ae2761,0,b'While developing a suite of tools for statistical machine translation research, we recognized the need for a visualization tool that would allow researchers to examine and evaluate specific word correspondences generated by a translation system.'),

Then we count the number of documents that contain exactly N segments, where N=1, 2, 3, 4, 5.

In [22]:
def get_document_text(doc):
    text_string=" ".join([str(sentence.text) for sentence in doc.sentences])
    return text_string


def rank_by_matched_segments(document_breakdown_map):
    for n in [5,4,3,2,1]:
        print("Below are one or two document examples that contain exactly "+str(n)+" segments\n")
        showed_examples=0
        count=0
        for ind,docid in enumerate(document_breakdown_map.keys()):
            if len(document_breakdown_map[docid].keys())==n:
                count+=1
                if showed_examples<2:
                    showed_examples+=1
                    print(docid,": ", get_document_text(document_breakdown_map[docid][list(document_breakdown_map[docid].keys())[0]][0].get_parent().get_parent()))
                    print("Extracted segments\n\n",docid,document_breakdown_map[docid],"\n\n")
        print("Total count is "+str(count)+"\n\n=================\n")
        
# rank_by_matched_segments(document_breakdown_map)
        

Then we want to some visualization of extracted Span as highlighted text on Document.

Color notation: <b style="color:orange;">Background</b> <b style="color:pink;">Purpose</b> <b style="color:green;">Mechanism</b> <b style="color:purple;">Method</b> <b style="color:blue;">Findings</b>

In [23]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
    
    
def print_colored_text(docid,document_breakdown_map=document_breakdown_map):
    
    color_mapping={"Background":"orange","Purpose":"pink","Mechanism":"green","Method":"purple","Finding":"blue"}
    this_document=document_breakdown_map[docid][list(document_breakdown_map[docid].keys())[0]][0].get_parent().get_parent()
    document_text=get_document_text(this_document)

    added_segment=[]
    print("This document has "+str(len(document_breakdown_map[docid]))+" spans")
    for segment in document_breakdown_map[docid]:
        spans=document_breakdown_map[docid][segment]
#         print(segment)
        for ind,span in enumerate(spans):
#             print(span.__dict__)
            this_span=document_breakdown_map[docid][segment][ind].__dict__[list(document_breakdown_map[docid][segment][ind].__dict__)[6]]
            span_sentence_text=this_span.sentence.text
            span_text=str(span_sentence_text[this_span.char_start:(this_span.char_end+1)])
#             print("span_text is "+span_text)
            document_text=document_text.replace(span_text.strip(),"<font color='"+color_mapping[segment]+"'>"+span_text.strip()+"</font>")

    printmd(document_text)

# docid='5834868625ff05a97b013016'
# print_colored_text(docid)   


In [29]:
# test_background_cands=extract_and_display(non_comma_dict_background_matcher,Background,"Background",document_breakdown_map=test_doc_breakdown_map,selected_split=2,is_print=False)
# test_purpose_cands=extract_and_display(non_comma_dict_purpose_matcher,Purpose,"Purpose",document_breakdown_map=test_doc_breakdown_map,selected_split=2,is_print=False)
# test_mechanism_cands=extract_and_display(non_comma_dict_mechanism_matcher,Mechanism,"Mechanism",document_breakdown_map=test_doc_breakdown_map,selected_split=2,is_print=False)
# test_method_cands=extract_and_display(non_comma_dict_method_matcher,Method,"Method",document_breakdown_map=test_doc_breakdown_map,selected_split=2,is_print=False)
# test_finding_cands=extract_and_display(non_comma_dict_finding_matcher,Finding,"Finding",document_breakdown_map=test_doc_breakdown_map,selected_split=2,is_print=False)

print_colored_text('cscw18assessment',document_breakdown_map=test_doc_breakdown_map)

This document has 2 spans


Security requirements analysis depends on how well-trained analysts perceive security risk, understand the impact of various vulnerabilities, and mitigate threats. When systems are composed of multiple machines, configurations, and software components that interact with each other, risk perception must account for the composition of security requirements. In this paper, we report on how changes to security requirements affect analysts risk perceptions and their decisions about how to modify the requirements to reach adequate security levels. <font color='purple'>We conducted two user surveys of 174 participants wherein participants assess security levels across 64 factorial vignettes.</font> <font color='purple'><font color='blue'>We analyzed the survey results using multi-level modeling to test for the effect of security requirements composition on participants' overall security adequacy ratings and on their ratings of individual requirements.</font></font> We accompanied this analysis with grounded analysis of elicited requirements aimed at lowering the security risk. <font color='blue'>Our results suggest that requirements composition affects experts' adequacy ratings on security requirements.</font> In addition, we identified three categories of requirements modifications, called refinements, replacements and reinforcements, and we measured how these categories compare with overall perceived security risk. Finally, we discuss the future impact of our work in security requirements assessment practice.